In [ ]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import talib
from app import *

In [ ]:
# Load data
df = pd.read_pickle(S.data_file)

In [ ]:
df.head()

In [ ]:
%%timeit
talib.EMA(df['AAPL']['Close'],2)

In [ ]:
%%timeit
df['AAPL'][['Close']].apply(lambda c: talib.EMA(c, 2))

In [ ]:
df1 = df.xs('Close',level=1,axis=1)
df1.head()

In [ ]:
%%timeit
for c in df1.columns:
    talib.EMA(df1[c],2)

In [ ]:
%%timeit
df.apply(lambda c: talib.EMA(c.values, 2))

In [ ]:
%%timeit
df.apply(lambda c: ta.ema(c, 2))
# ta.sma(df["Close"], length=10)

In [ ]:
      def ema_multiple_periods(df ,ema_list = [8, 21, 34, 55, 89]):
        #initializes an empty data frame with empty multi-index columns
        multi_index_cols = pd.MultiIndex.from_product([[],[]])
        ema = pd.DataFrame(index=df.index,columns=multi_index_cols)
        
        for period in ema_list:
            col = '{}{}'.format('ema', period)
            ema = ema.join(df.apply(lambda c: talib.EMA(c.values, period)).rename(columns={'Close': col}))
        return ema

        ema_df = ema_multiple_periods(df1 ,ema_list = [8, 21, 34, 55, 89])
# df.apply(lambda c: talib.EMA(c.values, 2))
# df.columns
# df.loc[:,('SYK','XXX')] = df.loc[:,('SYK','Open')]

In [ ]:
Snapshot()

ema

In [ ]:
    # To DO
    # Calculate EMAs
    # Calculate SMAs
    # Calculate MACDs
    
    # TODO parametrize function arguments for variable length, e.g. a list
    def check_rule_ema_positively_stacked(ema8,ema21,ema34,ema55, ema89):
        """Returns true if each previous ema is higher than the next
            input must be in order of check e.g. 1st > 2nd > 3rd...

         Args:
             ema8  (float): numeric values

         Returns:
             True or False
        """
        rules = [ema8  > ema21,
                ema21 > ema34,
                ema34 > ema55,
                ema55 > ema89]
        return all(rules)
    
    def check_rule_sma_fast_over_slow_crossover(sma_fast, sma_slow):
        """Returns True if sma_fast is now higher than sma_slow 
           in the most recent period and not in the previous

        Args:
            sma_fast (series): time series
            sma_slow (series): time series

        Returns:
            [True or False]
        """
        rules = [sma_fast[-1] > sma_slow[-1],
             sma_fast[-2] < sma_slow[-2],]
        return all(rules)    

In [ ]:
# check_rule_ema_positively_stacked(5,4,3,2,1)
check_rule_sma_fast_over_slow_crossover(df['AAPL']['Close'],df['AAPL']['Close'])


In [ ]:
    def macd_filter(**kwargs):
        df = kwargs['df']
        direction = kwargs['direction']
        n_periods = kwargs['n_periods']
        
        macd, macdsignal, macdhist = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
        macd_cross = (macd - macdsignal) > 0
        macd_bull_signal = macd_cross > macd_cross.shift(periods=1)
        macd_bear_signal = macd_cross < macd_cross.shift(periods=1)
        if direction == 'bull':
            result = macd_bull_signal[-n_periods:].max() > 0 #check for cross in last n periods
        elif direction == 'bear':
            result = macd_bull_signal[-n_periods:].max() < 0 #check for cross in last n periods
        return True if result else False
    
    def rsi_filter(**kwargs):  
        df = kwargs['df']
        rsi = talib.RSI(df['Close'], timeperiod=14)
        result = rsi[-1] < 30
        return True if result else False

In [ ]:
data=df['AAPL']
macd_filter(df=data, direction='bull',n_periods=5)

In [ ]:
data=df['AAPL']
rsi_filter(df=data)

In [ ]:
def ttm_squeeze(**kwargs):
        df = kwargs['df']
        sq = pd.DataFrame()
        sq['20sma'] = df['Close'].rolling(window=20).mean()
        sq['stddev'] = df['Close'].rolling(window=20).std()
        sq['lower_band'] = sq['20sma'] - (2 * sq['stddev'])
        sq['upper_band'] = sq['20sma'] + (2 * sq['stddev'])
        sq['TR'] = abs(df['High'] - df['Low'])
        sq['ATR'] = sq['TR'].rolling(window=20).mean()
        sq['lower_keltner'] = sq['20sma'] - (sq['ATR'] * 1.5)
        sq['upper_keltner'] = sq['20sma'] + (sq['ATR'] * 1.5)

        def in_squeeze(_):
            return _['lower_band'] > _['lower_keltner'] and \
                   _['upper_band'] < _['upper_keltner']

        sq['squeeze_on'] = sq.apply(in_squeeze, axis=1)

        if 'in_the_squeeze' in kwargs.keys():
            if kwargs['in_the_squeeze']:
                return sq

        if sq.iloc[-3]['squeeze_on'] and not sq.iloc[-1]['squeeze_on']:
            # is coming out the squeeze
            return True
        return False

    def in_the_squeeze(self, **kwargs):
        # is in the squeeze
        sq = self.ttm_squeeze(df=kwargs['df'], in_the_squeeze=True)
        result = True if sq.iloc[-1]['squeeze_on'] else False
        return result

In [ ]:
atr = talib.ATR(data['High'],data['Low'],data['Close'], timeperiod=20)
upperband, middleband, lowerband = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
macd, macdsignal, macdhist = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
sma_fast = talib.SMA(df['Close'], timeperiod=25)
sma_slow = talib.SMA(df['Close'], timeperiod=50)

In [ ]:
def ema_multiple_periods(df ,ema_list = [8, 21, 34, 55, 89]):
    #initializes an empty data frame with empty multi-index columns
    multi_index_cols = pd.MultiIndex.from_product([[],[]])
    ema = pd.DataFrame(index=df.index,columns=multi_index_cols)

    for period in ema_list:
        col = '{}{}'.format('ema', period)
        ema = ema.join(df.apply(lambda c: talib.EMA(c.values, period)).rename(columns={'Close': col}))
    return ema

def sma_multiple_periods(df , sma_list = [25, 50]):
    #initializes an empty data frame with empty multi-index columns
    multi_index_cols = pd.MultiIndex.from_product([[],[]])
    sma = pd.DataFrame(index=df.index,columns=multi_index_cols)

    for period in sma_list:
        col = '{}{}'.format('sma', period)
        sma = sma.join(df.apply(lambda c: talib.SMA(c.values, period)).rename(columns={'Close': col}))
    return sma

   


In [ ]:
data = df.xs('Close', level=1, axis=1, drop_level=False) 
ema_df = ema_multiple_periods(data,ema_list = [8, 21, 34, 55, 89])

data = df.xs('Close', level=1, axis=1, drop_level=False) 
sma_df = sma_multiple_periods(data ,sma_list = [25,50])

In [ ]:
def sma_multiple_periods(df , sma_list = [25, 50]):
    #initializes an empty data frame with empty multi-index columns
    multi_index_cols = pd.MultiIndex.from_product([[],[]])
    sma = pd.DataFrame(index=df.index,columns=multi_index_cols)

    for period in sma_list:
        col = '{}{}'.format('sma', period)
        sma = sma.join(df.apply(lambda c: talib.SMA(c.values, period)).rename(columns={'Close': col}))
    return sma

In [1]:
import talib
import pandas as pd
df = pd.read_pickle('datasets/data.pkl')

def atr_apply_fun(x, timeperiod=20):
    return talib.ATR(x[x.name]['High'], x[x.name]['Low'], x[x.name]['Close'], timeperiod)

def sma_apply_fun(x, timeperiod=20):
    return talib.SMA(x[x.name]['Close'], timeperiod)
    
def ema_apply_fun(x, timeperiod=20):
    return talib.EMA(x[x.name]['Close'], timeperiod)

def rsi_apply_fun(x, timeperiod=14):
    return talib.RSI(x[x.name]['Close'], timeperiod)

def cci_apply_fun(x, timeperiod=14):
    return talib.CCI(x[x.name]['High'], x[x.name]['Low'], x[x.name]['Close'], timeperiod)

def macd_apply_fun(x, fastperiod=12, slowperiod=26, signalperiod=9):
    macd, macdsignal, macdhist = talib.MACD(x[x.name]['Close'], fastperiod, slowperiod, signalperiod)
    return macd

def macdsignal_apply_fun(x, fastperiod=12, slowperiod=26, signalperiod=9):
    macd, macdsignal, macdhist = talib.MACD(x[x.name]['Close'], fastperiod, slowperiod, signalperiod)
    return macdsignal

def macdhist_apply_fun(x, fastperiod=12, slowperiod=26, signalperiod=9):
    macd, macdsignal, macdhist = talib.MACD(x[x.name]['Close'], fastperiod, slowperiod, signalperiod)
    return macdhist

def bband_upper_apply_fun(x, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0):
    upperband, middleband, lowerband = talib.BBANDS(x[x.name]['Close'], timeperiod, nbdevup, nbdevdn, matype)
    return upperband

def bband_lower_apply_fun(x, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0):
    upperband, middleband, lowerband = talib.BBANDS(x[x.name]['Close'], timeperiod, nbdevup, nbdevdn, matype)
    return lowerband

def bband_middle_apply_fun(x, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0):
    upperband, middleband, lowerband = talib.BBANDS(x[x.name]['Close'], timeperiod, nbdevup, nbdevdn, matype)
    return middleband


In [2]:

indicators_dict = {}

ema_list = [8, 21, 34, 55, 89]
for timeperiod in ema_list:
    k = '{}{}'.format('ema', timeperiod)
    indicators_dict[k] = df.groupby(level=0, axis=1).apply(lambda x: ema_apply_fun(x, timeperiod))

sma_list = [10, 20, 25, 50]
for timeperiod in sma_list:
    k = '{}{}'.format('sma', timeperiod)
    indicators_dict[k] = df.groupby(level=0, axis=1).apply(lambda x: sma_apply_fun(x, timeperiod))

indicators_dict['atr'] = df.groupby(level=0, axis=1).apply(lambda x: atr_apply_fun(x, timeperiod=20))
indicators_dict['rsi'] = df.groupby(level=0, axis=1).apply(lambda x: rsi_apply_fun(x, timeperiod=14))
indicators_dict['cci'] = df.groupby(level=0, axis=1).apply(lambda x: cci_apply_fun(x, timeperiod=14))

indicators_dict['macd'] = df.groupby(level=0, axis=1).apply(lambda x: macd_apply_fun(x, fastperiod=12, slowperiod=26, signalperiod=9))
indicators_dict['macdsignal'] = df.groupby(level=0, axis=1).apply(lambda x: macdsignal_apply_fun(x, fastperiod=12, slowperiod=26, signalperiod=9))
indicators_dict['macdhist'] = df.groupby(level=0, axis=1).apply(lambda x: macdhist_apply_fun(x, fastperiod=12, slowperiod=26, signalperiod=9))

indicators_dict['bband_upper'] = df.groupby(level=0, axis=1).apply(lambda x: bband_upper_apply_fun(x, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0))
indicators_dict['bband_lower'] = df.groupby(level=0, axis=1).apply(lambda x: bband_lower_apply_fun(x, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0))
indicators_dict['bband_middle'] = df.groupby(level=0, axis=1).apply(lambda x: bband_middle_apply_fun(x, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0))

indicators_dict['lower_keltner'] = indicators_dict['sma20'] - (indicators_dict['atr']*1.5)
indicators_dict['upper_keltner'] = indicators_dict['sma20'] + (indicators_dict['atr']*1.5)

In [3]:
indicators_dict.keys()

dict_keys(['ema8', 'ema21', 'ema34', 'ema55', 'ema89', 'sma10', 'sma20', 'sma25', 'sma50', 'atr', 'rsi', 'cci', 'macd', 'macdsignal', 'macdhist', 'bband_upper', 'bband_lower', 'bband_middle', 'lower_keltner', 'upper_keltner'])

In [4]:
# pd.concat(indicators_dict, axis=1).swaplevel(0,1,axis=1).to_csv('~/downloads/postprocess.csv')
# s = ['AAPL']
# plt.figure(figsize=(20,10))
# plt.plot(indicators_dict['lower_keltner'][s])
# plt.plot(indicators_dict['upper_keltner'][s])
# plt.plot(indicators_dict['sma20'][s])
# plt.plot(df['AAPL']['Close'])

In [5]:
def check_rule_ema_stacked(indicators_dict):
    ema_periods = sorted([int(k.replace('ema','')) for k in indicators_dict.keys() if 'ema' in k])
    
    df = 1 #initialize a df as True
    for i in range((len(ema_periods)-1)):
        k1 = '{}{}'.format('ema',ema_periods[i])
        k2 = '{}{}'.format('ema',ema_periods[i+1])
        df1 = indicators_dict[k1] > indicators_dict[k2]
        df = df*(df1.astype(int))
    return df.astype(bool)

def check_rule_sma_crossover(indicators_dict, fast=25, slow=50, bull=True):
    k_fast = '{}{}'.format('sma', fast)
    k_slow = '{}{}'.format('sma', slow)
    if bull:
        df1 = indicators_dict[k_fast] > indicators_dict[k_slow]
        df2 = indicators_dict[k_fast].shift(1) < indicators_dict[k_slow].shift(1)
    else:
        df1 = ~(indicators_dict[k_fast] > indicators_dict[k_slow])
        df2 = ~(indicators_dict[k_fast].shift(1) < indicators_dict[k_slow].shift(1))
    return df1.astype(int).mul(df2.astype(int),axis=1).astype(bool)

def check_rule_in_squeeze(indicators_dict):
    df1 = (indicators_dict['bband_lower'] > indicators_dict['lower_keltner'])
    df2 = (indicators_dict['bband_upper'] < indicators_dict['upper_keltner'])
    df =  df1.astype(int).mul(df2.astype(int), axis=0)
    return df.astype(bool)

def check_rule_coming_out_squeeze(squeeze_df):
    df1 = (indicators_dict['bband_lower'] > indicators_dict['lower_keltner'])
    df2 = (indicators_dict['bband_upper'] < indicators_dict['upper_keltner'])
    return df1.astype(int).mul(df2.astype(int), axis=0).astype(bool)

def check_rule_rsi(indicators_dict, bull = True):
    overbought = indicators_dict['rsi'] > 70
    oversold = indicators_dict['rsi'] < 30
    return oversold if bull else overbought


def check_rule_macd_crossover(indicators_dict, bull=True):
    if bull:
        df1 = indicators_dict['macd'] > indicators_dict['macdsignal']
        df2 = indicators_dict['macd'].shift(1) < indicators_dict['macdsignal'].shift(1)
    else:
        df1 = ~(indicators_dict['macd'] > indicators_dict['macdsignal'])
        df2 = ~(indicators_dict['macd'].shift(1) < indicators_dict['macdsignal'].shift(1))
    return df1.astype(int).mul(df2.astype(int),axis=1).astype(bool)



In [6]:
signals = pd.DataFrame()
signals.index.set_names('symbol',inplace=True)

df = check_rule_in_squeeze(indicators_dict)
signals.loc[:,'ttm_in_squeeze'] = df.iloc[-1]
signals.loc[:,'ttm_coming_out_squeeze'] = (~df.iloc[-1] ) & (df.iloc[-3])

df = check_rule_ema_stacked(indicators_dict)
signals.loc[:,'ema_positively_stacked'] = df.iloc[-1]

df = check_rule_sma_crossover(indicators_dict, fast=25, slow=50, bull=True)
signals.loc[:,'sma_crossover_bull'] = df.iloc[-1]

df = check_rule_rsi(indicators_dict, bull=True)
signals.loc[:,'rsi_oversold'] = df.iloc[-1]

df = check_rule_rsi(indicators_dict, bull=False)
signals.loc[:,'rsi_overbought'] = df.iloc[-1]

df = check_rule_macd_crossover(indicators_dict, bull=True)
signals.loc[:,'macd_cross_bull'] = df.iloc[-1]

df = check_rule_macd_crossover(indicators_dict, bull=False)
signals.loc[:,'macd_cross_bear'] = df.iloc[-1]

signals

,ttm_in_squeeze,ttm_coming_out_squeeze,ema_positively_stacked,sma_crossover_bull,rsi_oversold,rsi_overbought,macd_cross_bull,macd_cross_bear
symbol,,,,,,,,
A,False,False,False,False,False,False,False,False
AAL,False,False,True,False,False,False,False,False
AAP,False,False,True,False,False,False,False,False
AAPL,False,False,False,False,False,False,False,False
ABBV,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...
YUM,True,False,False,False,False,False,False,False
ZBH,True,False,False,False,False,False,False,False
ZBRA,False,False,True,False,False,False,False,False


In [79]:
checkbox_a1 = True
checkbox_a2 = True
checkbox_a3 = False
checkbox_a4 = False
checkbox_a5 = True
checkbox_a6 = False
checkbox_b1 = False
checkbox_b2 = False
checkbox_b3 = False





symbol
NOC     True
PAYX    True
dtype: bool

In [87]:
signals.index
filter_results = pd.DataFrame(index=signals.index)
filter_results.loc[:,'initialized'] = 1

In [94]:
filter_results.mul(signals['ttm_in_squeeze', axis=1])

SyntaxError: invalid syntax (<ipython-input-94-450307c39203>, line 1)

In [140]:
filter_results
filter_results[:]=1


symbol
A       1
AAL     1
AAP     1
AAPL    1
ABBV    1
       ..
YUM     1
ZBH     1
ZBRA    1
ZION    1
ZTS     1
Name: scanner, Length: 505, dtype: int64

In [143]:
filter_results.mul(signals['ttm_in_squeeze'], axis=0)

symbol
A       0
AAL     0
AAP     0
AAPL    0
ABBV    1
       ..
YUM     1
ZBH     1
ZBRA    0
ZION    0
ZTS     0
Length: 505, dtype: int64

In [130]:
new_df = new_df.mul(signals['ttm_in_squeeze'],axis=0)

In [131]:
new_df = new_df.mul(signals['ema_positively_stacked'],axis=0)

,init
symbol,
A,0
AAL,0
AAP,0
AAPL,0
ABBV,1
...,...
YUM,0
ZBH,0
ZBRA,0
